Code cleanup. Need to 
1. remove repeated sentences due to vtt formatting
2. some words are concatenated together (helloBut--> hello But, swimming in a pool.We--> swimming in a pool. We)

In [ ]:
import glob
import json
import regex as re
import os
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk import FreqDist
from nltk.corpus.reader import *
import nltk
# nltk.download('stopwords')
# nltk.download('vader_lexicon')
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
import pandas as pd
import mplcursors
import seaborn as sns

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
positive = ['positive', 'negative']
bad_test=['bad', 'worse', 'worst']
emojis=[':D', 'XD', ':p', ':(']
slang = ['ffs', 'smh']

negative = ['i tripped and fell and broke a bone', 'she misses home', 'ffs']
positive = ['life is beautiful', "that's so exciting", "i couldn't wish for anything better- this is perfect"]
neutral = ['i folded the clothes.', 'he checked the time']

post_twenty_fourteen=['pog', 'hangry'] #2019, 2018, 2017
boosterWords=['very nice', 'VERY nice', 'VERY NICE!']






negative = ['i tripped and fell and broke a bone', 'she misses home', 'ffs']

def sia_sentiment(listOfWords):
    for sentence in listOfWords:
        sid = SIA()
        ss = sid.polarity_scores(sentence)
        for k in sorted(ss)[:1]:
            print('{0}: {1}'.format(sentence, ss[k]), end='')
        print()

sia_sentiment(negative)

In [ ]:
with open('../channelHolisticInfo.json', 'r') as j:
    channelHolisticInfo = json.loads(j.read())
channelHolisticInfoDf = pd.DataFrame.from_dict(channelHolisticInfo)
channelHolisticInfoDf.drop('60', inplace = True)
channelHolisticInfoDf.index = range(len(channelHolisticInfoDf))
channelHolisticInfoDf

In [ ]:
LIST_OF_CHANNEL_IDS = channelHolisticInfoDf['channelId'].tolist()
LIST_OF_CHANNEL_NAMES = channelHolisticInfoDf['channelName'].tolist()
LIST_OF_VIDEO_COUNT = channelHolisticInfoDf['videoCount'].tolist()
print(len(LIST_OF_VIDEO_COUNT), len(LIST_OF_VIDEO_COUNT), len(LIST_OF_VIDEO_COUNT))

In [ ]:
def generateListOfSentences(filepath):
    with open(filepath, 'r') as f:
        lines = f.readlines()
        strippedLines = [line.strip() for line in lines if line.strip()] 
        duplicatedLinesList = []
        for index in range(len(strippedLines)):
            duplicatedLinesList.append(strippedLines[index])
    return [i for n, i in enumerate(duplicatedLinesList) if i not in duplicatedLinesList[:n]]
# print(generateListOfSentences("UCAlxwHEm1bWWYoe5VCSQYEg/txt/_9KytbCHaXU.txt"))

def generateListOfWords(listOfSentences):
    all_stopwords = list(stopwords.words('english'))
    cleanedListOfWords = []
    for words in listOfSentences:
        word = re.split('[?."(),!) ]', words)
        for wo in word:
            if wo !='' and wo not in all_stopwords:
                cleanedListOfWords.append(wo)
    return cleanedListOfWords

def flattenList(list):
    flat_list = [item for sublist in list for item in sublist]
    return flat_list

#wordCloud[channelId] = all words from all .txt files with no stopwords
#sentenceCloud[channelId] = all sentences from all .txt files.

wordCloud = {}
sentenceCloud = {}
for channelId in LIST_OF_CHANNEL_IDS:
    listOfSentences = []
    listOfWords = []
    print(channelId, "has ", len(glob.glob(channelId+"/txt/*.txt")), "files to process.")
    for txtFilePath in glob.glob(channelId+"/txt/*.txt"):
        listOfSentences.append(generateListOfSentences(txtFilePath))
    flattenedListOfSentences = flattenList(listOfSentences) #<--
    flattendListOfWords = generateListOfWords(flattenedListOfSentences)

    sentenceCloud[channelId.split('/')[0]] = flattenedListOfSentences
    wordCloud[channelId.split('/')[0]] = flattendListOfWords
    print(channelId.split('/')[0], ": ", len(flattenedListOfSentences), "sentences total")
    print(channelId.split('/')[0], ": ", len(flattendListOfWords), "words total")

    # with open(channelId.split('/')[0]+"_wordCloud.txt", 'w') as f:
    #     json.dump(flattendListOfWords, f)
    # with open(channelId.split('/')[0]+"_sentenceCloud.txt", 'w') as f:
    #     json.dump(flattenedListOfSentences, f)

# Holistically

In [ ]:
#General sentiment by words spoken by youtubers

from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
sia = SIA()
results = []

print(len(wordCloud.keys()))
for channelId in LIST_OF_CHANNEL_IDS:
    for words in wordCloud[channelId]:
        pol_score = sia.polarity_scores(words)
        pol_score['words']=words
        results.append(pol_score)
pd.set_option('display.max_columns', None, 'max_colwidth', None)
df = pd.DataFrame.from_records(results)
df['label']=0
df.loc[df['compound'] > 0.10, 'label'] = 1
df.loc[df['compound'] < -0.10, 'label'] = -1

import seaborn as sns
import matplotlib.pyplot as plt
fix, ax = plt.subplots(figsize=(8,8))
counts = df.label.value_counts(normalize=True)*100
sns.barplot(x=counts.index, y=counts, ax=ax)
ax.set_xticklabels(['Negative', 'Neutral', 'Positive'])
ax.set_ylabel(["Percentage"])
plt.show()

df.head()

In [ ]:
df_positive_negative = df.loc[df['label'] != 0]
df_positive_negative.head()

In [ ]:
print(df_positive_negative.label.value_counts())

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))
counts = df_positive_negative.label.value_counts(normalize=True)*100
sns.barplot(x=counts.index, y=counts, ax=ax)
ax.set_xticklabels(['Neg', 'Pos'])
ax.set_ylabel('Percentage')
plt.show()

Basically, compared to redditors, animation youtubers are more 'positive'.

Which is honestly not that surprising. Redditors are among the most realistic people out there and youtubers are generally out there to 'entertain'. 

# By channel and sentences

In [ ]:
#General sentiment by words spoken by youtubers

from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
sia = SIA()
results = []
for channelId in LIST_OF_CHANNEL_IDS:
    for sentence in sentenceCloud[channelId]:
        pol_score = sia.polarity_scores(sentence)
        pol_score['sentence']=sentence
        results.append(pol_score)
pd.set_option('display.max_columns', None, 'max_colwidth', None)
df = pd.DataFrame.from_records(results)
df['label']=0
df.loc[df['compound'] > 0.10, 'label'] = 1
df.loc[df['compound'] < -0.10, 'label'] = -1

import seaborn as sns
import matplotlib.pyplot as plt
fix, ax = plt.subplots(figsize=(8,8))
counts = df.label.value_counts(normalize=True)*100
sns.barplot(x=counts.index, y=counts, ax=ax)
ax.set_xticklabels(['Negative', 'Neutral', 'Positive'])
ax.set_ylabel(["Percentage"])
plt.show()

df.head()

# By channel by sentences

In [ ]:
sentenceCloud

In [ ]:
#General sentiment by words spoken by youtubers
from IPython.display import display
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
sia = SIA()
channelId_to_sentencedf = {}

for channelId in LIST_OF_CHANNEL_IDS:
    results = []
    for sentence in sentenceCloud[channelId]:
        pol_score = sia.polarity_scores(sentence)
        pol_score['sentence']=sentence
        results.append(pol_score)
    pd.set_option('display.max_columns', None, 'max_colwidth', None)
    df = pd.DataFrame.from_records(results)
    channelId_to_sentencedf[channelId] = df
df['label']=0
df.loc[df['compound'] > 0.10, 'label'] = 1
df.loc[df['compound'] < -0.10, 'label'] = -1

for channelId in LIST_OF_CHANNEL_IDS[8:12]:
    display(channelId_to_sentencedf[channelId][200:210])

In [ ]:
df_positive_negative = df.loc[df['label'] != 0]
df_positive_negative.head()

channel_df = channelId_to_sentencedf['UCnsem444vdU1HhS0mb2wwTA']
target_df = channel_df.loc[channel_df['compound'] > 0]
target_df


In [ ]:
top_20_pos_words = {}
top_20_neg_words = {}
scores= {}
for channelId in LIST_OF_CHANNEL_IDS:
    channel_df = channelId_to_sentencedf[channelId]
    target_df = channel_df.loc[channel_df['compound'] > 0]
    scores[channelId] = sum(target_df['compound'])/len(target_df)
channelName_to_no_neutral_scores = {}
for i in range(0, len(LIST_OF_CHANNEL_NAMES)):
    channelName_to_no_neutral_scores[LIST_OF_CHANNEL_NAMES[i]] = list(scores.values())[i]

avg_score=sum(channelName_to_no_neutral_scores.values())/len(LIST_OF_CHANNEL_NAMES)
print("avg score =", avg_score)
print(sorted(channelName_to_no_neutral_scores.items(), key=lambda item: item[1]))

In [ ]:
top_20_pos_words = {}
top_20_neg_words = {}
scores= {}
for channelId in LIST_OF_CHANNEL_IDS:
    target_df = channelId_to_sentencedf[channelId]
    positive_words = list(target_df.loc[target_df['pos']==1].sentence)
    positive_frequency = FreqDist(positive_words)
    pos_freq = positive_frequency.most_common(20)
    negative_words = list(target_df.loc[target_df['neg']==1].sentence)
    negative_frequency = FreqDist(negative_words)
    neg_freq = negative_frequency.most_common(20)
    top_20_pos_words[channelId] = pos_freq
    top_20_neg_words[channelId] = neg_freq
    pos_words = [str(p) for p in pos_freq]
    pos_words_str = ', '.join(pos_words)
    neg_words = [str(n) for n in neg_freq]
    neg_words_str = ', '.join(neg_words)
    # generateWordCloud(pos_words_str, neg_words_str)
    scores[channelId] = sum(target_df['compound'])/len(target_df)

print(len(scores), len(LIST_OF_CHANNEL_NAMES))

channelName_to_scores = {}
for i in range(0, len(LIST_OF_CHANNEL_NAMES)):
    channelName_to_scores[LIST_OF_CHANNEL_NAMES[i]] = list(scores.values())[i]

avg_score=sum(channelName_to_scores.values())/len(LIST_OF_CHANNEL_NAMES)
print("avg score =", avg_score)
print(sorted(channelName_to_scores.items(), key=lambda item: item[1]))

In [ ]:
sorted_channelName_to_scores = dict(sorted(channelName_to_scores.items(), key=lambda item: item[1]))
sctsdf = pd.DataFrame.from_dict({'channelName':sorted_channelName_to_scores.keys(), 'scores': sorted_channelName_to_scores.values()})
sctsdf

pd.DataFrame(sctsdf).to_csv("semantic_scores.csv")

In [ ]:
%matplotlib widget


In [ ]:
x_name = "Channel Name"
y_name = "Positivity"
tooltip_name = "positivity value"

x= sctsdf['channelName']
y= sctsdf['scores']
tt = sctsdf['scores']
fig, ax = plt.subplots(1, figsize=(12,6))
ax.set_title ('Semantic Scores of Each Channel')
fig.canvas.manager.set_window_title ('My Window Title')
sc = ax.scatter(x, y)
# Plot the average line
mean_x=np.arange(0,61,1)
mean_y=[avg_score for i in np.zeros_like(x)]
mean_line = ax.plot(mean_x,mean_y, label='Mean', linestyle='--')
plt.tick_params(
    axis='x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=False) # labels along the bottom edge are off
cursor = mplcursors.cursor(sc, hover=True)

plt.xlabel(x_name)
plt.ylabel(y_name)
plt.show()


In [ ]:
num_of_files = []
for channelId in LIST_OF_CHANNEL_IDS:
    num_of_files.append(len(os.listdir(channelId+"/txt/")))

cntvdf = pd.DataFrame.from_dict({'channelName':LIST_OF_CHANNEL_NAMES, 'number of transcripts collected': num_of_files, 'number of vids': LIST_OF_VIDEO_COUNT})

cntvdf

In [ ]:
representation = sum(cntvdf['number of transcripts collected'])/sum(cntvdf['number of vids'])
print(representation)

In [ ]:
#https://towardsai.net/p/data-science/how-when-and-why-should-you-normalize-standardize-rescale-your-data-3f083def38ff
subsPerYearSinceFirstUpload = channelHolisticInfoDf['subscribersPerYearSinceFirstUpload']

In [ ]:
zipped = zip(channelHolisticInfoDf.channelName, channelHolisticInfoDf.subscribersPerYearSinceFirstUpload)
sorted_channelName_to_rescaledSubs = dict(sorted(dict(zipped).items(), key=lambda item: item[1]))
sctrs = pd.DataFrame.from_dict({'channelName': sorted_channelName_to_rescaledSubs.keys(), 'rescaledSubCount': sorted_channelName_to_rescaledSubs.values()})
sctrs

In [ ]:
import math
x_name = "Channel"
y_name = "rescaledSubCount"
tooltip_name = "subCountPerYearSinceFirstUpload"

x= sctrs['channelName']
y= sctrs['rescaledSubCount']
y_log = [math.log(i) for i in sctrs['rescaledSubCount']]
tt = sctrs['rescaledSubCount']

fig, ax = plt.subplots(1, figsize=(12,6))
sc = ax.scatter(x, y)
ax.set_title ('Rescaled Subscriber Counts')

avg_rescaledSubcount= sum(y)/len(LIST_OF_CHANNEL_NAMES)
# Plot the average line
mean_x=np.arange(0,61,1)
mean_y=[avg_rescaledSubcount for i in np.zeros_like(x)]
mean_line = ax.plot(mean_x,mean_y, label='Mean', linestyle='--')
# mean_line = ax.plot(mean_x, mean_y, label='Mean', linestyle='--')
plt.tick_params(
    axis='x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=False) # labels along the bottom edge are off
cursor = mplcursors.cursor(sc, hover=True)

plt.xlabel(x_name)
plt.ylabel(y_name)
plt.show()

In [ ]:
semantic_df = pd.DataFrame.from_dict({'channelName': channelHolisticInfoDf['channelName'], 'rescaledSubCount': subsPerYearSinceFirstUpload, 'positivityScores': channelName_to_scores.values()})
def generate_plot(semantic_df):
    x_name = "averageCompoundScore"
    y_name = "rescaledSubCount"

    tooltip_name = "channelName"
    x= semantic_df['positivityScores']
    y= semantic_df['rescaledSubCount']

    tt = semantic_df[tooltip_name]
    fig, ax = plt.subplots(1, figsize=(12,6))
    sc = ax.scatter(x, y)

    avg_rescaledSubcount= sum(y)/len(LIST_OF_CHANNEL_NAMES)
    # Plot the average line
    mean_x=np.arange(0,33,1)
    mean_y=[avg_rescaledSubcount for i in np.zeros_like(x)]
    # mean_line = ax.plot(mean_x, mean_y, label='Mean', linestyle='--')

    annot = ax.annotate("", xy=(0,0), xytext=(20,20),textcoords="offset points",
                        bbox=dict(boxstyle="round", fc="w"),
                        arrowprops=dict(arrowstyle="->"))
    ax.set_title ('Rescaled Subscriber Counts v. Semantic Scores')

    annot.set_visible(False)

    def update_annot(details):
        pos = sc.get_offsets()[details["channelName"]]
        annot.xy = pos
        text = "{}, {}".format(" ".join(details["channelName"]), 
                            " ".join([details["positivityScores"]]))
        annot.set_text(text)
        annot.get_bbox_patch().set_alpha(0.4)

    def hover(event):
        vis = annot.get_visible()
        if event.inaxes == ax:
            cont, details = sc.contains(event)
            if cont:
                update_annot(details)
                annot.set_visible(True)
                fig.canvas.draw_idle()
            else:
                if vis:
                    annot.set_visible(False)
                    fig.canvas.draw_idle()

    cursor = mplcursors.cursor(sc, hover=True)
    cursor.connect(
    "add", lambda sel: sel.annotation.set_text(tt[sel.index]))
    plt.xlabel("semantic score")
    plt.ylabel("number of subscribers per year since first upload date")
    plt.show()

    m, b = np.polyfit(x, y, 1)
    plt.plot(x, m*x + b)
generate_plot(semantic_df)

x= 'rescaledSubCount'	
y= 'positivityScores'
result = scipy.stats.linregress(semantic_df[x], semantic_df[y])
print("no outliers removed: ", 'slope: {}, rvalue:{}, pvalue:{}'.format(result.slope, round(result.rvalue, 5), round(result.pvalue, 5)))

In [ ]:
import numpy as np
import pandas as pd
def detect_outlier(data_1):
    outliers=[]
    # threshold=3 means any data with a stdDev> 3 or <-3 is 0.3 of all data. Almost all of the data (99.7%) should be within 3 stddev from the mean
    threshold=3
    mean_1 = np.mean(data_1)
    std_1 =np.std(data_1)
    
    index =0
    for y in data_1:
        z_score= (y - mean_1)/std_1 
        if np.abs(z_score) > threshold:
            outliers.append((index, y))
        index+=1
    return outliers
print("===========rescaledSubs outlier===========")
for indexToScore in detect_outlier(subsPerYearSinceFirstUpload):
    print("{}:{}".format(channelHolisticInfoDf['channelName'][indexToScore[0]], indexToScore[1]))
print("===========semantic_scores outliers===========")
for indexToScore in detect_outlier(list(channelName_to_scores.values())):
    print("{}:{}".format(channelHolisticInfoDf['channelName'][indexToScore[0]], indexToScore[1]))

In [ ]:
semantic_df = pd.DataFrame.from_dict({'channelName': channelHolisticInfoDf['channelName'], 'rescaledSubCount': subsPerYearSinceFirstUpload, 'positivityScores': channelName_to_scores.values()})
semantic_df

In [ ]:
import scipy
import numpy as np

copy_without_rescaledSubs_outlier = semantic_df.drop(29)
copy_without_semantic_outlier = semantic_df.drop([28, 46, 58])
copy_without_rescaledSubs_and_semantic_outlier = semantic_df.drop([29, 28, 46, 58])
semantic_df_copy_without_rescaledSubs_outlier = pd.DataFrame.from_dict(json.loads(json.dumps(copy_without_rescaledSubs_outlier.to_dict())))
semantic_df_copy_without_semantic_outlier = pd.DataFrame.from_dict(json.loads(json.dumps(copy_without_semantic_outlier.to_dict())))
semantic_df_copy_without_rescaledSubs_and_semantic_outlier = pd.DataFrame.from_dict(json.loads(json.dumps(copy_without_rescaledSubs_and_semantic_outlier.to_dict())))

x= 'rescaledSubCount'	
y= 'positivityScores'
result = scipy.stats.linregress(semantic_df[x], semantic_df[y])
result_no_rescaled_outlier = scipy.stats.linregress(semantic_df_copy_without_rescaledSubs_outlier[x], semantic_df_copy_without_rescaledSubs_outlier[y])
result_no_semantic_outlier = scipy.stats.linregress(semantic_df_copy_without_semantic_outlier[x], semantic_df_copy_without_semantic_outlier[y])
result_no_rescaled_and_semantic_outliers = scipy.stats.linregress(semantic_df_copy_without_rescaledSubs_and_semantic_outlier[x], semantic_df_copy_without_rescaledSubs_and_semantic_outlier[y])

print("no outliers removed: ", 'rvalue:{}, pvalue:{}'.format(result.rvalue, result.pvalue))
print("removed positivity outlier: ", 'rvalue:{}, pvalue:{}'.format(result_no_rescaled_outlier.rvalue, result_no_rescaled_outlier.pvalue))
print("removed subCount outlier: ", 'rvalue:{}, pvalue:{}'.format(result_no_semantic_outlier.rvalue, result_no_semantic_outlier.pvalue))
print("removed both subCount and positivity outliers: ", 'rvalue:{}, pvalue:{}'.format(result_no_rescaled_and_semantic_outliers.rvalue, result_no_rescaled_and_semantic_outliers.pvalue))
print()
print("no outliers removed: ", scipy.stats.spearmanr(semantic_df[x_name], semantic_df[y_name], axis=0))
print("removed positivity outlier: ", scipy.stats.spearmanr(semantic_df_copy_without_semantic_outlier[x_name], semantic_df_copy_without_semantic_outlier[y_name], axis=0))
print("removed subCount outlier: ",  scipy.stats.spearmanr(semantic_df_copy_without_rescaledSubs_outlier[x_name], semantic_df_copy_without_rescaledSubs_outlier[y_name], axis=0))
print("removed both subCount and positivity outliers: ", scipy.stats.spearmanr(semantic_df_copy_without_rescaledSubs_and_semantic_outlier[x_name], semantic_df_copy_without_rescaledSubs_and_semantic_outlier[y_name], axis=0))


The p (or probability) value obtained from the calculator is a measure of how likely or probable it is that any observed correlation is due to chance. P-values range between 0 (0%) and 1 (100%). A p-value close to 1 suggests no correlation other than due to chance and that your null hypothesis assumption is correct.

So how exactly strong is the correlation? 
We use r-values
https://www.researchgate.net/post/What-does-R-square-Adjusted-R-and-R-indicate-in-terms-of-Multiple-Regression-Analysis

Simply put, R is the correlation between the predicted values and the observed values of Y. R square is the square of this coefficient and indicates the percentage of variation explained by your regression line out of the total variation. This value tends to increase as you include additional predictors in the model. Thus, one can artificially get higher  R square by increasing the number of Xs in the model. To penalize this effect, adjusted R square is used. When you compare models with their complexity, you should then rely on Adj R square. Predicted R square is another measure which addresses the issue of overfitting the data  and explain the prediction power for future observations. 

The p (or probability) value obtained from the calculator is a measure of how likely or probable it is that any observed correlation is due to chance. P-values range between 0 (0%) and 1 (100%). A p-value close to 1 suggests no correlation other than due to chance and that your null hypothesis assumption is correct.

Tbh, the results kind of suck. 
Need to test the null hypothesis
The null hypothesis is a typical statistical theory which suggests that **no statistical relationship and significance exists in a set of given single observed variable**, between two sets of observed data and measured phenomena.

In [ ]:
def adjR(x, y, degree):
    results = {}
    coeffs = np.polyfit(x, y, degree)
    p = np.poly1d(coeffs)
    yhat = p(x)
    ybar = np.sum(y)/len(y)
    ssreg = np.sum((yhat-ybar)**2)
    sstot = np.sum((y - ybar)**2)
    results['r_squared'] = 1- (((1-(ssreg/sstot))*(len(y)-1))/(len(y)-degree-1))

    return results

for i in range(5):
    print(adjR(semantic_df[x], semantic_df[y], i))

! https://opentextbc.ca/researchmethods/chapter/understanding-null-hypothesis-testing/
A crucial step in null hypothesis testing is finding the likelihood of the sample result if the null hypothesis were true. This probability is called the p value. A low p value means that the sample result would be unlikely if the null hypothesis were true and leads to the rejection of the null hypothesis. A high p value means that the sample result would be likely if the null hypothesis were true and leads to the retention of the null hypothesis. But how low must the p value be before the sample result is considered unlikely enough to reject the null hypothesis? In null hypothesis testing, this criterion is called α (alpha) and is almost always set to .05. If there is less than a 5% chance of a result as extreme as the sample result if the null hypothesis were true, then the null hypothesis is rejected. When this happens, the result is said to be statistically significant. If there is greater than a 5% chance of a result as extreme as the sample result when the null hypothesis is true, then the null hypothesis is retained. This does not necessarily mean that the researcher accepts the null hypothesis as true—only that there is not currently enough evidence to conclude that it is true. Researchers often use the expression “fail to reject the null hypothesis” rather than “retain the null hypothesis,” but they never use the expression “accept the null hypothesis.”

> The Misunderstood p Value

> The p value is one of the most misunderstood quantities in psychological research (Cohen, 1994)[1]. Even professional researchers misinterpret it, and it is not unusual for such misinterpretations to appear in statistics textbooks!

> The most common misinterpretation is that the p value is the probability that the null hypothesis is true—that the sample result occurred by chance. For example, a misguided researcher might say that because the p value is .02, there is only a 2% chance that the result is due to chance and a 98% chance that it reflects a real relationship in the population. But this is incorrect. The p value is really the probability of a result at least as extreme as the sample result if the null hypothesis were true. So a p value of .02 means that if the null hypothesis were true, a sample result this extreme would occur only 2% of the time.

> You can avoid this misunderstanding by remembering that the p value is not the probability that any particular hypothesis is true or false. Instead, it is the probability of obtaining the sample result if the null hypothesis were true.

In [ ]:
from ace.samples import wang04
x, y = wang04.build_sample_ace_problem_wang04(N=200)
print(len(x), len(x[0]), len(y))
from ace import model
myace = model.Model()
myace.build_model_from_xy(x, y)
myace.eval([0.1, 0.2, 0.5, 0.3, 0.5])

In [ ]:
from ace import model
x= 'rescaledSubCount'	
y= 'positivityScores'
myace = model.Model()
myace.build_model_from_xy([semantic_df[x]], semantic_df[y])
myace.eval([0.0001])